In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
mpl.rcParams['font.family'] = 'serif'
import os

In [2]:
current_folder = globals()['_dh'][0]
rootdir = os.path.dirname(os.path.dirname(current_folder))
indir_inter = os.path.join(rootdir, '_2_intermediate', 'data')
indir_prepro = os.path.join(rootdir, '_1_preprocessing', 'data', 'raw', 'gis')
outdir = os.path.join(rootdir, '_3_figures_tables', 'data')

In [3]:
data = pd.read_csv(os.path.join(indir_inter, '_F_dist_religion_bch10.csv'))[['iso',
                                                         'bch10',
                                                         'major_religion',
                                                         'district',
                                                         'immg_18',
                                                         'n_immg_18']]
data = data[data.major_religion.isin(['Christian', 'Muslim', 'Traditional'])].reset_index(drop=True)
data = data[~((pd.isnull(data.immg_18)) & (pd.isnull(data.n_immg_18)))]

#####################################################################################

data1980 = data[data.bch10.isin([1980, 1990])].reset_index(drop=True)
data1980 = data1980[~((data1980.iso.isin(['LBR','MLI','NGA','TGO'])) & 
                      (data1980.bch10==1980))].reset_index(drop=True)
data1980 = data1980[~((~(data1980.iso.isin(['LBR','MLI','NGA','TGO']))) & 
                      (data1980.bch10==1990))].reset_index(drop=True)
del data1980['bch10']

data1980.index = [data1980.iso, data1980.district, data1980.major_religion]
del data1980['iso'], data1980['district'], data1980['major_religion']
data1980 = data1980.unstack().reset_index(drop=False)
data1980.columns = [x[0] + x[1] for x in data1980.columns]

data1980['gap_cm'] = data1980.immg_18Christian - data1980.immg_18Muslim
data1980['gap_ct'] = data1980.immg_18Christian - data1980.immg_18Traditional

data1980['gap_cm_min10'] = data1980['gap_cm']
data1980['gap_ct_min10'] = data1980['gap_ct']
data1980.loc[(data1980.n_immg_18Christian < 10) | (data1980.n_immg_18Muslim < 10), 'gap_cm_min10'] = np.NaN
data1980.loc[(data1980.n_immg_18Christian < 10) | (data1980.n_immg_18Traditional < 10), 'gap_ct_min10'] = np.NaN
data1980 = data1980[['iso', 'district', 'gap_cm', 'gap_ct', 'gap_cm_min10', 'gap_ct_min10']]

data1980 = data1980[~((pd.isnull(data1980.gap_cm)) &
                      (pd.isnull(data1980.gap_ct)) &
                      (pd.isnull(data1980.gap_cm_min10)) &
                      (pd.isnull(data1980.gap_ct_min10)))]

cns = ['iso', 'district'] + [x + '_1980' for x in data1980.columns[2:]]
data1980.columns = cns
del data1980['iso']


#####################################################################################

data['x'] = data.immg_18 * data.n_immg_18
del data['bch10']
data = data.groupby([data.iso, data.district, data.major_religion]).sum()
# del data['district']
data = data.reset_index(drop=False)
data['immg_18'] = data['x'] / data['n_immg_18']
del data['x']

data.index = [data.iso, data.district, data.major_religion]
del data['iso'], data['district'], data['major_religion']
data = data.unstack().reset_index(drop=False)
data.columns = [x[0] + x[1] for x in data.columns]

data['gap_cm'] = data.immg_18Christian - data.immg_18Muslim
data['gap_ct'] = data.immg_18Christian - data.immg_18Traditional

data['gap_cm_min10'] = data['gap_cm']
data['gap_ct_min10'] = data['gap_ct']
data.loc[(data.n_immg_18Christian < 10) | (data.n_immg_18Muslim < 10), 'gap_cm_min10'] = np.NaN
data.loc[(data.n_immg_18Christian < 10) | (data.n_immg_18Traditional < 10), 'gap_ct_min10'] = np.NaN
data = data[['iso', 'district', 'gap_cm', 'gap_ct', 'gap_cm_min10', 'gap_ct_min10']]

data = data[~((pd.isnull(data.gap_cm)) &
              (pd.isnull(data.gap_ct)) &
              (pd.isnull(data.gap_cm_min10)) &
              (pd.isnull(data.gap_ct_min10)))]

In [4]:
data

,iso,district,gap_cm,gap_ct,gap_cm_min10,gap_ct_min10
0,BEN,1.0,0.094657,0.174525,0.094657,0.174525
1,BEN,2.0,0.139636,0.211991,0.139636,0.211991
2,BEN,3.0,0.000627,0.110247,0.000627,0.110247
3,BEN,4.0,0.318973,0.421053,0.318973,NaN
4,BEN,5.0,0.511052,0.594633,0.511052,0.594633
...,...,...,...,...,...,...
2281,ZMB,2753.0,NaN,0.071925,NaN,0.071925
2282,ZMB,2754.0,NaN,0.068753,NaN,0.068753
2283,ZMB,2755.0,NaN,0.076782,NaN,0.076782
2284,ZMB,2756.0,0.374133,-0.116190,NaN,-0.116190


In [5]:
ISO = sorted(list(set(data.iso)))

In [6]:
countries = gpd.read_file(os.path.join(indir_prepro, 'africa_countries', 'africa_countries.shp'))
afrregions = gpd.read_file(os.path.join(indir_prepro, 'districts_for_plotting', 'districts_for_plotting.shp'))[['iso', 'district', 'geometry']]
afrregions = afrregions[afrregions.iso.isin(ISO)].reset_index(drop=True).copy()
afrregions = pd.merge(afrregions, data, on=['iso', 'district'], how='left')
afrregions = pd.merge(afrregions, data1980, on='district', how='left')
africa = afrregions[afrregions.geometry!=None].copy(deep=True)

In [7]:
for col in africa.columns[3:]:
    africa[col] = np.round(africa[col], 2)

In [8]:
countries['iso'] = countries.iso3
for c in africa.columns:
    if c not in countries.columns:
        countries[c] = np.nan
countries = countries[[c for c in africa.columns]]
countries = countries[~countries.iso.isin(list(set(africa.iso)) + ['ESP', 'PRT', 'FRA', 'YEM', 'SHN', 'ATF', 'REU'])]

In [9]:
africa = pd.concat([africa, countries], axis=0, ignore_index=True)

In [10]:
africa

,iso,district,geometry,gap_cm,gap_ct,gap_cm_min10,gap_ct_min10,gap_cm_1980,gap_ct_1980,gap_cm_min10_1980,gap_ct_min10_1980
0,BEN,1.0,"POLYGON ((2.40452 11.67795, 2.42524 11.66394, ...",0.09,0.17,0.09,0.17,0.01,0.07,0.01,0.07
1,BEN,2.0,"POLYGON ((3.19362 10.90465, 3.19200 10.89730, ...",0.14,0.21,0.14,0.21,0.14,-0.12,0.14,NaN
2,BEN,3.0,"POLYGON ((3.25846 11.29882, 3.25769 11.29174, ...",0.00,0.11,0.00,0.11,-0.02,0.14,-0.02,NaN
3,BEN,4.0,"POLYGON ((2.84088 12.40599, 2.84306 12.40454, ...",0.32,0.42,0.32,NaN,0.94,1.00,NaN,NaN
4,BEN,5.0,"POLYGON ((3.08195 11.97610, 3.08603 11.97523, ...",0.51,0.59,0.51,0.59,0.61,NaN,0.61,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2997,ZWE,NaN,"POLYGON ((29.86384 -15.61295, 29.87364 -15.615...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2998,SDN,NaN,"MULTIPOLYGON (((35.64075 23.09098, 35.64022 23...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2999,SDN,NaN,"MULTIPOLYGON (((37.27043 20.76363, 37.26722 20...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3000,SDN,NaN,"MULTIPOLYGON (((38.79701 18.54980, 38.79571 18...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
print('number of mappable districts for which we observe upward IM for both Christians and Muslims aged 14-18                                                               :', africa[~pd.isnull(africa.gap_cm)].drop_duplicates(subset=['district']).shape[0])
print('number of mappable districts for which we observe upward IM for both Christians and Traditionals aged 14-18                                                          :', africa[~pd.isnull(africa.gap_ct)].drop_duplicates(subset=['district']).shape[0])
print('number of mappable districts for which we observe upward IM for both Christians and Muslims aged 14-18 with a minimum of 10 observations                             :', africa[~pd.isnull(africa.gap_cm_min10)].drop_duplicates(subset=['district']).shape[0])
print('number of mappable districts for which we observe upward IM for both Christians and Traditionals aged 14-18 with a minimum of 10 observations                        :', africa[~pd.isnull(africa.gap_ct_min10)].drop_duplicates(subset=['district']).shape[0])

print()

print('number of mappable districts for which we observe upward IM for both Christians and Muslims aged 14-18 and born in 1980s/1990s                                       :', africa[~pd.isnull(africa.gap_cm_1980)].drop_duplicates(subset=['district']).shape[0])
print('number of mappable districts for which we observe upward IM for both Christians and Traditionals aged 14-18 and born in 1980s/1990s                                  :', africa[~pd.isnull(africa.gap_ct_1980)].drop_duplicates(subset=['district']).shape[0])
print('number of mappable districts for which we observe upward IM for both Christians and Muslims aged 14-18 with and born in 1980s/1990s a minimum of 10 observations     :', africa[~pd.isnull(africa.gap_cm_min10_1980)].drop_duplicates(subset=['district']).shape[0])
print('number of mappable districts for which we observe upward IM for both Christians and Traditionals aged 14-18 and born in 1980s/1990s with a minimum of 10 observations:', africa[~pd.isnull(africa.gap_ct_min10_1980)].drop_duplicates(subset=['district']).shape[0])

number of mappable districts for which we observe upward IM for both Christians and Muslims aged 14-18                                                               : 1861
number of mappable districts for which we observe upward IM for both Christians and Traditionals aged 14-18                                                          : 1322
number of mappable districts for which we observe upward IM for both Christians and Muslims aged 14-18 with a minimum of 10 observations                             : 1299
number of mappable districts for which we observe upward IM for both Christians and Traditionals aged 14-18 with a minimum of 10 observations                        : 819

number of mappable districts for which we observe upward IM for both Christians and Muslims aged 14-18 and born in 1980s/1990s                                       : 1731
number of mappable districts for which we observe upward IM for both Christians and Traditionals aged 14-18 and born in 1980s/1990s         

In [12]:
missing_kwds = dict(color='white', label='no Data')

titles = ['upward IM\nC-M gap', 'upward IM\nC-T gap', 'upward IM\nC-M gap', 'upward IM\nC-T gap']
for idvar, variable in enumerate(['gap_cm_1980', 'gap_ct_1980', 'gap_cm_min10_1980', 'gap_ct_min10_1980']):
    title = titles[idvar]

    f, ax = plt.subplots(1,1, figsize=(10, 10))
    africa.plot(ax=ax, 
            column=variable,
            scheme='user_defined',
            classification_kwds={'bins':[-0.8,-0.6,-0.4,-0.2,0,0.2,0.4,0.6,0.8,1]},
            cmap = 'RdBu',
            edgecolor='k',
            linewidth=0.1,
            legend=True, 
            missing_kwds=missing_kwds)
    ax.axis('off')
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0.15, 0.3, 0.2, 0.2))
    for txt in leg.texts:
        txti = txt.get_text().replace('-0.88', '-1.00').replace('-inf', '-1.00').replace('-0.94', '-1.00')
        txti = re.sub('\.([0-9])0', r'.\1',  txti.replace(', ', ' - '))
        txt.set_text(txti)
    leg.set_title(title, prop={'size': 12, 'weight': 'bold'})
    
    f.savefig(outdir + f'/_4_heatmap_{variable}.pdf', bbox_inches='tight')
    plt.close(f)